In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=116a1cff88f3d9b0cabf869e8aa4778d2fc3dc7583f60387e559d99f185d8785
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
import numpy as np 
import math
import pandas as pd 
import nltk 
import sklearn 
import re
from string import punctuation 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer

#from nltk import word_tokenize,ISRIStemmer
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
reviews2 = pd.read_csv('/content/drive/MyDrive/merged2.csv',sep=',',encoding='utf-8')

In [ ]:
reviews2.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,conversation_id,created_at,date,time,timezone,user_id,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,related
0,0,0,0,1487936212853796866,1487936212853796866,2022-01-31 05:20:12 India Standard Time,2022-01-31,05:20:12,530,3323082688,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,1
1,3,3,3,1487934149671460872,1487934149671460872,2022-01-31 05:12:00 India Standard Time,2022-01-31,05:12:00,530,231726084,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,1
2,4,4,4,1487931199716798470,1487931199716798470,2022-01-31 05:00:17 India Standard Time,2022-01-31,05:00:17,530,2858863684,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,1
3,5,5,5,1487931194452983808,1487931194452983808,2022-01-31 05:00:15 India Standard Time,2022-01-31,05:00:15,530,734538503770505216,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,1
4,6,6,6,1487930263711543296,1487930263711543296,2022-01-31 04:56:33 India Standard Time,2022-01-31,04:56:33,530,740567406473543680,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,1


In [ ]:
reviews2['tweet'][0]

'Long Covid, sintomi e causa: nuovo studio su\xa0polmoni  https://t.co/uwQMLtEBZU'

In [ ]:
reviews2['tweet'][548]

'@AlexBazzaro Speriamo non sia come detto da Montagner una pandemia dei vaccinati!!già ora sintomi dubbi vengono associati al long covid e mai al vaccino……'

In [ ]:
reviews2.shape

(1074, 40)

In [ ]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=cd9f0308eac14cfd42d24319ceb00488815b6bac0b373e7da9d676cb232002a2
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
from stop_words import get_stop_words

stop_words = get_stop_words('it')
stop_words = get_stop_words('italian')

In [ ]:
print(stop_words)

['a', 'abbia', 'abbiamo', 'abbiano', 'abbiate', 'ad', 'adesso', 'agl', 'agli', 'ai', 'al', 'all', 'alla', 'alle', 'allo', 'allora', 'altre', 'altri', 'altro', 'anche', 'ancora', 'avemmo', 'avendo', 'avere', 'avesse', 'avessero', 'avessi', 'avessimo', 'aveste', 'avesti', 'avete', 'aveva', 'avevamo', 'avevano', 'avevate', 'avevi', 'avevo', 'avrai', 'avranno', 'avrebbe', 'avrebbero', 'avrei', 'avremmo', 'avremo', 'avreste', 'avresti', 'avrete', 'avrà', 'avrò', 'avuta', 'avute', 'avuti', 'avuto', 'c', 'che', 'chi', 'ci', 'coi', 'col', 'come', 'con', 'contro', 'cui', 'da', 'dagl', 'dagli', 'dai', 'dal', 'dall', 'dalla', 'dalle', 'dallo', 'degl', 'degli', 'dei', 'del', 'dell', 'della', 'delle', 'dello', 'dentro', 'di', 'dov', 'dove', 'e', 'ebbe', 'ebbero', 'ebbi', 'ecco', 'ed', 'era', 'erano', 'eravamo', 'eravate', 'eri', 'ero', 'essendo', 'faccia', 'facciamo', 'facciano', 'facciate', 'faccio', 'facemmo', 'facendo', 'facesse', 'facessero', 'facessi', 'facessimo', 'faceste', 'facesti', 'facev

In [ ]:
stop_words=stopwords.words('english')+list(punctuation)

In [ ]:
reviews2['tweet'].fillna(' ', inplace=True)

In [ ]:
reviews2['tweet']=reviews2['tweet'].fillna(0)

In [ ]:
def preparing(text):
    #ps = PorterStemmer()
    tokens= text.split()
    words=[w.lower() for w in tokens]
    words= [w for w in words if  w not in stop_words]
    return words

In [ ]:
data_words=[]    
for i in range(0,1074):
    
    #if math.isnan(reviews2['cleaned'][i])== False:
     if reviews2['tweet'][i] != 0 :
                data_words.append(preparing(reviews2['tweet'][i]))

In [ ]:
print(data_words[:1])

[['long', 'covid,', 'sintomi', 'causa:', 'nuovo', 'studio', 'polmoni', 'https://t.co/uwqmltebzu']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['long', 'covid,', 'sintomi', 'causa:_nuovo_studio', 'polmoni', 'https://t.co/uwqmltebzu']


In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

In [ ]:
print(data_words_bigrams[0])

['long', 'covid,', 'sintomi', 'causa:_nuovo', 'studio', 'polmoni', 'https://t.co/uwqmltebzu']


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [ ]:
id2word[0]

'causa:_nuovo'

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('causa:_nuovo', 1),
  ('covid,', 1),
  ('https://t.co/uwqmltebzu', 1),
  ('long', 1),
  ('polmoni', 1),
  ('sintomi', 1),
  ('studio', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.082*"sintomi" + 0.073*"long" + 0.042*"covid" + 0.015*"covid," + '
  '0.014*"dopo" + 0.010*"studio" + 0.008*"covid." + 0.008*"bambini" + '
  '0.007*"-" + 0.006*"mesi"'),
 (1,
  '0.015*"sintomi" + 0.013*"covid:" + 0.011*"viene_dubbio" + '
  '0.010*"essere_stati" + 0.010*"alcuni_potrebbero" + 0.010*"considerare_ciò" '
  '+ 0.010*"covid." + 0.007*"tanti" + 0.007*"quali" + 0.007*"galli"'),
 (2,
  '0.009*"#longcovid" + 0.007*"sintomi" + 0.006*"fatto" + 0.004*"grave" + '
  '0.004*"stato" + 0.004*"dice" + 0.004*"longcovid" + 0.004*"caso" + '
  '0.003*"vaccini" + 0.003*"preso"'),
 (3,
  '0.015*"#longcovid" + 0.014*"sintomi" + 0.005*"casi" + 0.004*"parte" + '
  '0.004*"molto" + 0.004*"milioni" + 0.004*"soffre" + 0.004*"persone" + '
  '0.004*"stato" + 0.003*"#covid"')]


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -8.488182634226849

Coherence Score:  0.3821078330441108


In [ ]:
# Visualize the topics
#pyLDAvis.enable_notebook()

#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#vis
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.234442  0.028184       1        1  45.682767
2      0.106158  0.068702       2        1  19.536147
3      0.085924  0.078462       3        1  18.912983
1      0.042359 -0.175347       4        1  15.868103, topic_info=                   Term        Freq       Total Category  logprob  loglift
3                  long  664.000000  664.000000  Default  30.0000  30.0000
13                covid  374.000000  374.000000  Default  29.0000  29.0000
5               sintomi  848.000000  848.000000  Default  28.0000  28.0000
1                covid,  132.000000  132.000000  Default  27.0000  27.0000
128                dopo  127.000000  127.000000  Default  26.0000  26.0000
...                 ...         ...         ...      ...      ...      ...
3                  long   17.627677  664.864758   Topic4  -5.1673  -1.7893
1125  potrebbero_essere    8.150125   17.703199   Topic4  -5.9387   1.0651
149              alcuni    8.222632   23.277713   Topic4  -5.9299   0.8003
54           #longcovid    8.383378   95.959589   Topic4  -5.9105  -0.5968
232            pazienti    7.529315   42.236215   Topic4  -6.0179   0.1164

[201 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
6359      4  0.972105  "ci_così
8         1  0.626545    #covid
8         3  0.362737    #covid
3444      1  0.112427   #covid,
3444      3  0.899412   #covid,
...     ...       ...       ...
667       1  0.992353     virus
1003      3  0.918603         x
825       4  0.933254         |
2581      4  0.945814         –
2180      3  0.856564         👇

[203 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2])